# python-can CAN over serial

[```python-can```](https://python-can.readthedocs.io/en/2.1.0/index.html) includes a [CAN over Serial] module.

This allows prototyping and testing with inexpensive Arduino devices before switching to another module with minimal code changes.


The packet over serial



https://python-can.readthedocs.io/en/2.1.0/interfaces/serial.html

|                | Length (Byte) | Data type               | Byte order    | Description                                   |
|----------------|---------------|-------------------------|---------------|-----------------------------------------------|
| Start of frame | 1             | Byte                    | -             | Must be 0xAA                                  |
| Timestamp      | 4             | Unsigned 4 byte integer | Little-Endian | Usually s, ms or µs since start of the device |
| DLC            | 1             | Unsigned 1 byte integer | Little-Endian | Length in byte of the payload                 |
| Arbitration ID | 4             | Unsigned 4 byte integer | Little-Endian | -                                             |
| Payload        | 0 - 8         | Byte                    | -             | -                                             |
| End of frame   | 1             | Byte                    | -             | Must be 0xBB                                  |

In [17]:
arduino_template_str = """
unsigned long BAUD = {{ baud }};
unsigned long DELAY = {{ delay }};
char ARBITRATION_ID = {{ arbitration_id }};

unsigned long time;

unsigned char counter = 0;

// Arduino Setup
void setup() {
  Serial.begin(BAUD);
    // Start of frame
  Serial.write(0xAA);
// Timestamp
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
// DLC
  Serial.write(0x01);
// Arbitration ID
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0xFF);
// Payload
  Serial.write(counter);
// End of frame
  Serial.write(0xBB);
    
  counter++;
}

// Arduino Main Loop
void loop() {
   time = millis();
// Start of frame
  Serial.write(0xAA);
// Timestamp
for(char shift=0;shift<32;shift+=8) {
  Serial.write((unsigned long)time>>shift&0b11111111);
}
// DLC
  Serial.write(0x01);
// Arbitration ID
for(char shift=0;shift<32;shift+=8) {
  Serial.write(((unsigned long)ARBITRATION_ID>>shift)&0b11111111);
}
// Payload
  Serial.write(counter);
// End of frame
  Serial.write(0xBB);
    
  counter++;
  delay(DELAY);
}
"""

In [18]:
import jinja2

In [19]:
arduino_template = jinja2.Template(arduino_template_str)

In [22]:
print(arduino_template.render(baud=115200, delay=1000, arbitration_id = 12))


unsigned long BAUD = 115200;
unsigned long DELAY = 1000;
char ARBITRATION_ID = 12;

unsigned long time;

unsigned char counter = 0;

// Arduino Setup
void setup() {
  Serial.begin(BAUD);
    // Start of frame
  Serial.write(0xAA);
// Timestamp
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
// DLC
  Serial.write(0x01);
// Arbitration ID
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0x00);
  Serial.write(0xFF);
// Payload
  Serial.write(counter);
// End of frame
  Serial.write(0xBB);
    
  counter++;
}

// Arduino Main Loop
void loop() {
   time = millis();
// Start of frame
  Serial.write(0xAA);
// Timestamp
for(char shift=0;shift<32;shift+=8) {
  Serial.write((unsigned long)time>>shift&0b11111111);
}
// DLC
  Serial.write(0x01);
// Arbitration ID
for(char shift=0;shift<32;shift+=8) {
  Serial.write(((unsigned long)ARBITRATION_ID>>shift)&0b11111111);
}
// Payload
  Serial.write(counter);
// End of frame
  Serial.write(0xBB);
    
 

In [288]:
import subprocess

stdout = subprocess.check_output(["make", "upload"]).decode()
print(stdout)

-------------------------
Arduino.mk Configuration:
- [AUTODETECTED]       CURRENT_OS = LINUX 
- [AUTODETECTED]       ARDUINO_DIR = /usr/share/arduino 
- [COMPUTED]           ARDMK_DIR = /projects/arduino_IO/arduino_make (relative to Common.mk)
- [AUTODETECTED]       ARDUINO_VERSION = 105 
- [DEFAULT]            ARCHITECTURE =  
- [DEFAULT]            ARDMK_VENDOR = arduino 
- [AUTODETECTED]       ARDUINO_PREFERENCES_PATH = /mnt/ubuntu1604_2/home/jed/.arduino/preferences.txt 
- [AUTODETECTED]       ARDUINO_SKETCHBOOK = /mnt/ubuntu1604_2/home/jed/sketchbook (from arduino preferences file)
- [BUNDLED]            AVR_TOOLS_DIR = /usr/share/arduino/hardware/tools/avr (in Arduino distribution)
- [COMPUTED]           ARDUINO_LIB_PATH = /usr/share/arduino/libraries (from ARDUINO_DIR)
- [COMPUTED]           ARDUINO_VAR_PATH = /usr/share/arduino/hardware/arduino//variants (from ARDUINO_DIR)
- [COMPUTED]           BOARDS_TXT = /usr/share/arduino/hardware/arduino//boards.txt (from ARDUINO_DIR)
- 

In [289]:
import re
r = re.compile("(/dev/ttyUSB[\d]+)")
r.findall(stdout)

['/dev/ttyUSB2', '/dev/ttyUSB2', '/dev/ttyUSB2']

In [5]:
import can
can.__version__

'2.1.0'

In [6]:
bus = can.interface.Bus(bustype="serial",
                        channel="/dev/ttyUSB2", bitrate=115200)

In [7]:
bus.channel_info

'Serial interface: /dev/ttyUSB2'

In [8]:
packet = bus.recv(timeout=2)
packet

can.Message(timestamp=15.001, is_remote_frame=False, extended_id=True, is_error_frame=False, arbitration_id=0xc, dlc=1, data=[0x10])

In [9]:
packet = bus.recv()
packet

can.Message(timestamp=16.002, is_remote_frame=False, extended_id=True, is_error_frame=False, arbitration_id=0xc, dlc=1, data=[0x11])

In [10]:
packet.arbitration_id

12

In [273]:
packet.dlc

1

In [274]:
packet.timestamp

15.001

In [275]:
packet.is_error_frame

False

In [276]:
packet.error_state_indicator

False

In [277]:
packet.timestamp.real

15.001

In [278]:
bus.recv()

can.Message(timestamp=16.002, is_remote_frame=False, extended_id=True, is_error_frame=False, arbitration_id=0x36, dlc=1, data=[0x11])

In [284]:
for _ in range(20):
    packet = bus.recv(timeout=1000)
    if packet is None:
        continue
    else:
        break
    print(packet.timestamp)

In [285]:
packet

can.Message(timestamp=43.005, is_remote_frame=False, extended_id=True, is_error_frame=False, arbitration_id=0x36, dlc=1, data=[0x2c])

In [286]:
packet.arbitration_id.

54